In [8]:
import numpy as np
import os
import pandas as pd
import arcpy

#import arcgis libraries for Spatially-Enabled DataFrames
from arcgis.gis import *
gis = GIS()

In [27]:
working_directory = os.getcwd()
dataTAZ_directory = os.path.join(working_directory,"data\TAZ")
intrmed_directory = os.path.join(working_directory,"intermediate")
results_directory = os.path.join(working_directory,"results"     )

print(working_directory)

tazcentroids_shp = r"E:\GitHub\TDM-College-Enrollment-v9\TAZ\USTM_TAZ_2021_09_22_Centroids.shp";

#results
csv_tazzip = os.path.join(results_directory, r"taz20210922_zipcode.csv")

e:\GitHub\TAZ-to-ZIPCode


In [5]:
sde_filename = "agrc_opensgid"

#make only once
arcpy.CreateDatabaseConnection_management (working_directory, sde_filename, "POSTGRESQL", "opensgid.agrc.utah.gov", "DATABASE_AUTH", "agrc", "agrc", "SAVE_USERNAME", "opensgid")

<Result 'e:\\GitHub\\TAZ-to-ZIPCode\\agrc_opensgid.sde'>

In [6]:
zipcodearea_layername = "opensgid.boundaries.zip_code_areas"
zipcode = sde_filename + ".sde\\" + zipcodearea_layername
print(arcpy.Describe(zipcode).shapeType)

sdfZipCode = pd.DataFrame.spatial.from_featureclass(zipcode)
sdfZipCode

Polygon


In [21]:
sdfTAZCentroids = pd.DataFrame.spatial.from_featureclass(tazcentroids_shp)
sdfTAZCentroids

,FID,TAZID,SA_TAZID,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,...,TAZID_V21B,SATAZ_V21B,COTAZ_V21B,SUBID_V21B,TAZID_V21A,SATAZ_V21A,COTAZ_V21A,SUBID_V21A,ORIG_FID,SHAPE
0,0,43,3,3,1003,0,440.680556,440.680556,1.00,357308.618265,...,43,0,1013,0,43,0,1013,0,0,"{""x"": 357308.61828827206, ""y"": 4240407.8890121..."
1,1,45,5,5,1005,0,245.424579,245.424579,1.00,356023.333451,...,45,0,1015,0,45,0,1015,0,1,"{""x"": 356023.33349711914, ""y"": 4239022.5529349..."
2,2,46,6,6,1006,0,675.848293,675.848293,1.00,357530.306637,...,46,0,1016,0,46,0,1016,0,2,"{""x"": 357530.3066365821, ""y"": 4239070.37353264..."
3,3,55,15,15,1015,0,550.767119,534.244106,0.97,356277.254346,...,55,0,1025,0,55,0,1025,0,3,"{""x"": 356277.2543912772, ""y"": 4234918.19385224..."
4,4,50,10,10,1010,0,176.207617,176.207617,1.00,355998.119155,...,50,0,1020,0,50,0,1020,0,4,"{""x"": 355998.11920056865, ""y"": 4237044.8358770..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9810,9810,9615,120,120,21120,5,18.530419,18.530419,1.00,317587.467931,...,8618,119,21119,5,1241,0,21033,0,9810,"{""x"": 317587.4679099042, ""y"": 4175419.62035609..."
9811,9811,9634,139,139,21139,5,104.810861,79.656255,0.76,318283.808626,...,8636,137,21137,5,1259,0,21051,0,9811,"{""x"": 318283.8086040877, ""y"": 4174693.22996878..."
9812,9812,9569,74,74,21074,5,11.750705,11.750705,1.00,319032.243956,...,8572,73,21073,5,1226,0,21018,0,9812,"{""x"": 319032.24393162597, ""y"": 4179828.5837255..."
9813,9813,9793,298,298,21298,5,1011.540449,920.501809,0.91,341575.123412,...,8795,296,21296,5,1462,0,21254,0,9813,"{""x"": 341718.9014499998, ""y"": 4191679.73782058..."


In [22]:
sdfTAZCentroids_ZipCode = sdfTAZCentroids.spatial.join(sdfZipCode)
sdfTAZCentroids_ZipCode

,FID,TAZID,SA_TAZID,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,...,COTAZ_V21A,SUBID_V21A,ORIG_FID,SHAPE,index_right,xid,countynbr,name,symbol,zip5
0,0,43,3,3,1003,0,440.680556,440.680556,1.00,357308.618265,...,1013,0,0,"{""x"": 357308.61828827206, ""y"": 4240407.8890121...",203,204,1,BEAVER,5,84713
1,1,45,5,5,1005,0,245.424579,245.424579,1.00,356023.333451,...,1015,0,1,"{""x"": 356023.33349711914, ""y"": 4239022.5529349...",203,204,1,BEAVER,5,84713
2,2,46,6,6,1006,0,675.848293,675.848293,1.00,357530.306637,...,1016,0,2,"{""x"": 357530.3066365821, ""y"": 4239070.37353264...",203,204,1,BEAVER,5,84713
3,3,55,15,15,1015,0,550.767119,534.244106,0.97,356277.254346,...,1025,0,3,"{""x"": 356277.2543912772, ""y"": 4234918.19385224...",203,204,1,BEAVER,5,84713
4,4,50,10,10,1010,0,176.207617,176.207617,1.00,355998.119155,...,1020,0,4,"{""x"": 355998.11920056865, ""y"": 4237044.8358770...",203,204,1,BEAVER,5,84713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9810,9665,9500,5,5,21005,5,16771.166796,16771.166796,1.00,249570.017554,...,21145,0,9665,"{""x"": 249570.01756355353, ""y"": 4182116.6528390...",110,111,11,MODENA,2,84753
9811,9598,9821,326,326,21326,5,2675.544250,2675.544250,1.00,328405.123755,...,21245,0,9598,"{""x"": 328405.1237556217, ""y"": 4187011.49080289...",202,203,11,SUMMIT,2,84772
9812,9608,9831,336,336,21336,5,24845.539473,10683.581973,0.43,333493.279678,...,21240,0,9608,"{""x"": 333493.2796602249, ""y"": 4180601.23978496...",202,203,11,SUMMIT,2,84772
9813,9617,9826,331,331,21331,5,715.142241,693.687974,0.97,326061.163793,...,21224,0,9617,"{""x"": 326061.1637930721, ""y"": 4183325.71482209...",202,203,11,SUMMIT,2,84772


In [30]:
dfTAZZip = sdfTAZCentroids_ZipCode[['CO_TAZID','zip5']]
dfTAZZip.to_csv(csv_tazzip,index=False)
dfTAZZip

,CO_TAZID,zip5
0,1003,84713
1,1005,84713
2,1006,84713
3,1015,84713
4,1010,84713
...,...,...
9810,21005,84753
9811,21326,84772
9812,21336,84772
9813,21331,84772
